# Sistema de Microrred Híbrida con RL - Google Colab

Este notebook implementa un sistema híbrido de optimización de microrredes que combina:
- **Random Forest** para predicción de demanda y generación renovable
- **Reinforcement Learning (PPO)** para optimización del despacho de energía
- **Simulación de microrred** con diésel, solar, eólica y baterías

**Objetivo**: Minimizar costos operativos en microrredes de Zonas No Interconectadas (ZNI)

## Instrucciones para Google Colab:
1. Ejecuta todas las celdas en orden
2. En la celda 2, sube el archivo `datos_microrred_sc_p.csv`
3. El entrenamiento puede tomar varios minutos

## 1. Instalación de Librerías

In [ ]:
# CELDA 1: Instalaciones para Google Colab
!pip install stable-baselines3[extra] gym pandas scikit-learn numpy matplotlib shimmy torch

import gym
from gym import spaces
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from stable_baselines3 import PPO, DQN
from stable_baselines3.common.callbacks import EvalCallback
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías instaladas y importadas correctamente.")
print("📊 Versiones:")
print(f"- NumPy: {np.__version__}")
print(f"- Pandas: {pd.__version__}")
print(f"- Gym: {gym.__version__}")

## 2. Carga de Datos


In [ ]:
# CELDA 2: Carga de Datos para Google Colab
from google.colab import files
import io

print("📁 Por favor, sube el archivo 'datos_microrred_sc_p.csv'")
print("💡 Si no tienes el archivo, puedes usar los datos sintéticos de la siguiente celda")

# Subir el archivo de datos
uploaded = files.upload()

# Leer el archivo subido
df = None
for filename in uploaded.keys():
    print(f'✅ Archivo {filename} subido correctamente')
    df = pd.read_csv(io.BytesIO(uploaded[filename]))
    break

if df is not None:
    # Asegurar que el DataFrame esté ordenado por tiempo
    df['Tiempo'] = pd.to_datetime(df['Tiempo'])
    df = df.sort_values('Tiempo').reset_index(drop=True)
    
    # Crear features de tiempo si no existen
    if 'Hora' not in df.columns:
        df['Hora'] = df['Tiempo'].dt.hour
        df['Dia_Semana'] = df['Tiempo'].dt.dayofweek
    if 'Mes' not in df.columns:
        df['Mes'] = df['Tiempo'].dt.month
    
    print(f"✅ Datos cargados: {len(df)} filas")
    print("\n📋 Primeras 5 filas:")
    print(df.head())
    
    # Verificar columnas necesarias
    required_cols = ['Demanda', 'Solar_Gen', 'Eolica_Gen', 'Hora', 'Dia_Semana', 'Temperatura', 'Radiacion_Solar', 'Velocidad_Viento']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        print(f"⚠️ Columnas faltantes: {missing_cols}")
        print("💡 Usando datos sintéticos en su lugar...")
        df = None
else:
    print("⚠️ No se subió ningún archivo. Usando datos sintéticos...")
    df = None

# Solo hacer análisis si df no es None
if df is not None:
    # 1. Estadísticas Descriptivas por Tipo de Generación
    print("📈 === ESTADÍSTICAS DESCRIPTIVAS ===\n")
    gen_stats = pd.DataFrame({
        'Solar_Gen': {
            'Media (kW)': df['Solar_Gen'].mean(),
            'Máximo (kW)': df['Solar_Gen'].max(),
            'Mínimo (kW)': df['Solar_Gen'].min(),
            'Desv. Estándar (kW)': df['Solar_Gen'].std(),
            'Capacidad Factor (%)': (df['Solar_Gen'].mean() / df['Solar_Gen'].max() * 100) if df['Solar_Gen'].max() > 0 else 0,
            'Total Generado (MWh)': df['Solar_Gen'].sum() / 1000
        },
        'Eolica_Gen': {
            'Media (kW)': df['Eolica_Gen'].mean(),
            'Máximo (kW)': df['Eolica_Gen'].max(),
            'Mínimo (kW)': df['Eolica_Gen'].min(),
            'Desv. Estándar (kW)': df['Eolica_Gen'].std(),
            'Capacidad Factor (%)': (df['Eolica_Gen'].mean() / df['Eolica_Gen'].max() * 100) if df['Eolica_Gen'].max() > 0 else 0,
            'Total Generado (MWh)': df['Eolica_Gen'].sum() / 1000
        }
    }).T

    print(gen_stats.round(2))
    print("\n")

    # 2. Análisis de Patrones Temporales
    print("⏰ === ANÁLISIS DE PATRONES TEMPORALES ===\n")

    # Patrón por hora del día
    print("📊 Generación Promedio por Hora del Día:")
    hourly_pattern = pd.DataFrame({
        'Solar': df.groupby('Hora')['Solar_Gen'].mean(),
        'Eolica': df.groupby('Hora')['Eolica_Gen'].mean()
    })
    print(hourly_pattern.round(2))
    print("\n")

    # Patrón por mes
    print("📅 Generación Promedio por Mes:")
    monthly_pattern = pd.DataFrame({
        'Solar': df.groupby('Mes')['Solar_Gen'].mean(),
        'Eolica': df.groupby('Mes')['Eolica_Gen'].mean()
    })
    print(monthly_pattern.round(2))
    print("\n")

    # 3. Correlaciones
    print("🔗 === ANÁLISIS DE CORRELACIONES ===\n")
    correlation_cols = ['Solar_Gen', 'Eolica_Gen', 'Radiacion_Solar', 'Velocidad_Viento', 
                        'Temperatura', 'Hora', 'Demanda']
    corr_matrix = df[correlation_cols].corr()
    print("Correlación entre variables:")
    print(corr_matrix[['Solar_Gen', 'Eolica_Gen']].round(3))
    print("\n")

    # 4. Análisis de Variabilidad
    print("📉 === ANÁLISIS DE VARIABILIDAD ===\n")
    print("Coeficiente de Variación (CV = std/mean):")
    cv_solar = (df['Solar_Gen'].std() / df['Solar_Gen'].mean() * 100) if df['Solar_Gen'].mean() > 0 else 0
    cv_eolica = (df['Eolica_Gen'].std() / df['Eolica_Gen'].mean() * 100) if df['Eolica_Gen'].mean() > 0 else 0
    print(f"  Solar: {cv_solar:.2f}%")
    print(f"  Eólica: {cv_eolica:.2f}%")
    print("\n")

    # 5. Visualización del Análisis
    print("📊 Generando visualizaciones del análisis...")

    fig, axes = plt.subplots(3, 2, figsize=(16, 12))

    # Subplot 1: Patrón horario de generación
    axes[0, 0].plot(hourly_pattern.index, hourly_pattern['Solar'], 'o-', label='Solar', color='orange', linewidth=2)
    axes[0, 0].plot(hourly_pattern.index, hourly_pattern['Eolica'], 's-', label='Eólica', color='purple', linewidth=2)
    axes[0, 0].set_title('Patrón Horario de Generación', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Hora del Día')
    axes[0, 0].set_ylabel('Generación Promedio (kW)')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Subplot 2: Patrón mensual
    axes[0, 1].plot(monthly_pattern.index, monthly_pattern['Solar'], 'o-', label='Solar', color='orange', linewidth=2)
    axes[0, 1].plot(monthly_pattern.index, monthly_pattern['Eolica'], 's-', label='Eólica', color='purple', linewidth=2)
    axes[0, 1].set_title('Patrón Mensual de Generación', fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel('Mes')
    axes[0, 1].set_ylabel('Generación Promedio (kW)')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # Subplot 3: Distribución de generación solar
    axes[1, 0].hist(df['Solar_Gen'], bins=50, color='orange', alpha=0.7, edgecolor='black')
    axes[1, 0].axvline(df['Solar_Gen'].mean(), color='red', linestyle='--', linewidth=2, label=f'Media: {df["Solar_Gen"].mean():.1f} kW')
    axes[1, 0].set_title('Distribución de Generación Solar', fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel('Generación (kW)')
    axes[1, 0].set_ylabel('Frecuencia')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # Subplot 4: Distribución de generación eólica
    axes[1, 1].hist(df['Eolica_Gen'], bins=50, color='purple', alpha=0.7, edgecolor='black')
    axes[1, 1].axvline(df['Eolica_Gen'].mean(), color='red', linestyle='--', linewidth=2, label=f'Media: {df["Eolica_Gen"].mean():.1f} kW')
    axes[1, 1].set_title('Distribución de Generación Eólica', fontsize=12, fontweight='bold')
    axes[1, 1].set_xlabel('Generación (kW)')
    axes[1, 1].set_ylabel('Frecuencia')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    # Subplot 5: Generación vs. Radiación Solar
    axes[2, 0].scatter(df['Radiacion_Solar'], df['Solar_Gen'], alpha=0.3, s=1, color='orange')
    axes[2, 0].set_title('Relación: Radiación Solar vs. Generación Solar', fontsize=12, fontweight='bold')
    axes[2, 0].set_xlabel('Radiación Solar (W/m²)')
    axes[2, 0].set_ylabel('Generación Solar (kW)')
    axes[2, 0].grid(True, alpha=0.3)

    # Subplot 6: Generación vs. Velocidad del Viento
    axes[2, 1].scatter(df['Velocidad_Viento'], df['Eolica_Gen'], alpha=0.3, s=1, color='purple')
    axes[2, 1].set_title('Relación: Velocidad del Viento vs. Generación Eólica', fontsize=12, fontweight='bold')
    axes[2, 1].set_xlabel('Velocidad del Viento (m/s)')
    axes[2, 1].set_ylabel('Generación Eólica (kW)')
    axes[2, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print("✅ Análisis exploratorio completado\n")
else:
    print("⚠️ No hay datos disponibles para análisis. Se generarán datos sintéticos en la siguiente celda.\n")


## 3. Generación de Datos Sintéticos (Si es necesario)

In [ ]:
# CELDA 3: Generación de Datos Sintéticos (Solo si no se cargaron datos)
if df is None:
    print("🔄 Generando datos sintéticos...")
    
    # Parámetros del sistema
    DIESEL_P_MAX = 3500.0   # Capacidad Instalada (kW)
    BATT_CAP_MAX = 1500.0   # Capacidad Máxima de Batería (kWh)
    DIESEL_COST_KWH = 0.55  # Costo de Generación (USD/kWh)
    SOLAR_CAP_MW = 1.0      # Capacidad instalada solar (1000 kWp)
    EOLICA_CAP_KW = 500.0   # Capacidad instalada eólica (kW)
    
    # Parámetros para la Generación de Demanda
    ANOS_SIMULACION = 1  # Reducido para Colab
    DIESEL_P_PICO = 2800.0  # Pico de Demanda Máxima (kW)
    PERIODO_TOTAL = ANOS_SIMULACION * 8760  # Horas totales
    
    # Perfil de Carga Normalizado
    PERFIL_NORMALIZADO = [
        0.55, 0.40, 0.35, 0.30, 0.25, 0.32, 0.45, 0.60, 0.75, 0.80, 0.75, 0.70,
        0.65, 0.60, 0.55, 0.50, 0.55, 0.65, 0.75, 0.85, 0.95, 1.00, 0.90, 0.70
    ]
    
    # Factores de Ajuste Estacional
    FACTORES_ESTACIONALES = {
        1: 1.05, 2: 1.02, 3: 1.00, 4: 0.98, 5: 0.95, 6: 1.00,
        7: 1.05, 8: 1.02, 9: 0.98, 10: 0.95, 11: 1.00, 12: 1.05
    }
    
    # Crear DataFrame base
    tiempo = pd.date_range(start='2021-01-01 00:00:00', periods=PERIODO_TOTAL, freq='h')
    df = pd.DataFrame(index=tiempo)
    df['Hora'] = df.index.hour
    df['Dia_Semana'] = df.index.dayofweek
    df['Mes'] = df.index.month
    
    # Generación de la Demanda Sintética
    df['Demanda_Base'] = df['Hora'].apply(lambda h: PERFIL_NORMALIZADO[h] * DIESEL_P_PICO)
    df['Factor_Estacional'] = df['Mes'].map(FACTORES_ESTACIONALES)
    df['Demanda_Ajustada'] = df['Demanda_Base'] * df['Factor_Estacional']
    
    # Aplicar Ruido Aleatorio
    ruido = np.random.uniform(low=-0.03, high=0.03, size=PERIODO_TOTAL)
    df['Demanda'] = df['Demanda_Ajustada'] * (1 + ruido)
    df['Demanda'] = df['Demanda'].clip(lower=0)
    
    # Generación de Features Climáticos y Renovables
    # Radiación Solar
    df['Radiacion_Solar'] = 1000 * np.sin(np.pi * (df['Hora'] - 6) / 12).clip(lower=0)
    df['Radiacion_Solar'] += np.random.normal(0, 50, PERIODO_TOTAL) * (df['Radiacion_Solar'] > 0)
    df['Radiacion_Solar'] = df['Radiacion_Solar'].clip(lower=0)
    
    # Generación Solar
    df['Solar_Gen'] = SOLAR_CAP_MW * 1000.0 * (df['Radiacion_Solar'] / 1000.0) * 0.8
    df['Solar_Gen'] = df['Solar_Gen'].clip(upper=SOLAR_CAP_MW * 1000.0, lower=0)
    
    # Velocidad del Viento
    v_viento_base = 8.0 + 3.0 * np.cos(2 * np.pi * (df['Hora'] - 4) / 24)
    df['Velocidad_Viento'] = v_viento_base + np.random.normal(0, 1.5, PERIODO_TOTAL)
    df['Velocidad_Viento'] = df['Velocidad_Viento'].clip(lower=0)
    
    # Generación Eólica
    df['Eolica_Gen'] = np.where(df['Velocidad_Viento'] > 3,
                                EOLICA_CAP_KW * (df['Velocidad_Viento'] / 12)**3, 0)
    df['Eolica_Gen'] = df['Eolica_Gen'].clip(upper=EOLICA_CAP_KW)
    
    # Temperatura
    df['Temperatura'] = 28.0 + 3.0 * np.sin(np.pi * (df['Hora'] - 10) / 12).clip(lower=-1)
    df['Temperatura'] += np.random.normal(0, 1.0, PERIODO_TOTAL)
    df['Temperatura'] = df['Temperatura'].clip(lower=24.0)
    
    # DataFrame final
    df = df[['Demanda', 'Solar_Gen', 'Eolica_Gen', 'Hora', 'Dia_Semana', 'Temperatura', 'Radiacion_Solar', 'Velocidad_Viento']].copy()
    
    print(f"✅ Datos sintéticos generados: {len(df)} horas")
    print("\n📋 Primeras 5 filas:")
    print(df.head())
else:
    print("✅ Usando datos cargados del archivo CSV")

# Crear features temporales adicionales necesarias para el análisis
if 'Mes' not in df.columns:
    if isinstance(df.index, pd.DatetimeIndex):
        df['Mes'] = df.index.month
    elif 'Tiempo' in df.columns:
        df['Tiempo'] = pd.to_datetime(df['Tiempo'])
        df['Mes'] = df['Tiempo'].dt.month
    else:
        df.index = pd.date_range(start='2021-01-01', periods=len(df), freq='H')
        df['Mes'] = df.index.month

## 4. Análisis Exploratorio de Generación de Energía

In [ ]:
# CELDA 4: Análisis Exploratorio de Generación de Energía

print("📊 === ANÁLISIS DE GENERACIÓN DE ENERGÍA ===\n")

# Asegurar que tenemos el índice de tiempo correcto
if 'Tiempo' in df.columns:
    df['Tiempo'] = pd.to_datetime(df['Tiempo'])
    if not isinstance(df.index, pd.DatetimeIndex):
        df.set_index('Tiempo', inplace=True)
elif not isinstance(df.index, pd.DatetimeIndex):
    # Si no hay columna Tiempo, crear índices de tiempo
    df.index = pd.date_range(start='2021-01-01', periods=len(df), freq='H')

# Crear features temporales adicionales si no existen
if 'Mes' not in df.columns:
    df['Mes'] = df.index.month
if 'Dia' not in df.columns:
    df['Dia'] = df.index.day
if 'Estacion' not in df.columns:
    df['Estacion'] = df.index.month % 12 // 3 + 1  # 1=Invierno, 2=Primavera, etc.

# 1. Estadísticas Descriptivas por Tipo de Generación
print("📈 === ESTADÍSTICAS DESCRIPTIVAS ===\n")
gen_stats = pd.DataFrame({
    'Solar_Gen': {
        'Media (kW)': df['Solar_Gen'].mean(),
        'Máximo (kW)': df['Solar_Gen'].max(),
        'Mínimo (kW)': df['Solar_Gen'].min(),
        'Desv. Estándar (kW)': df['Solar_Gen'].std(),
        'Capacidad Factor (%)': (df['Solar_Gen'].mean() / df['Solar_Gen'].max() * 100) if df['Solar_Gen'].max() > 0 else 0,
        'Total Generado (MWh)': df['Solar_Gen'].sum() / 1000
    },
    'Eolica_Gen': {
        'Media (kW)': df['Eolica_Gen'].mean(),
        'Máximo (kW)': df['Eolica_Gen'].max(),
        'Mínimo (kW)': df['Eolica_Gen'].min(),
        'Desv. Estándar (kW)': df['Eolica_Gen'].std(),
        'Capacidad Factor (%)': (df['Eolica_Gen'].mean() / df['Eolica_Gen'].max() * 100) if df['Eolica_Gen'].max() > 0 else 0,
        'Total Generado (MWh)': df['Eolica_Gen'].sum() / 1000
    }
}).T

print(gen_stats.round(2))
print("\n")

# 2. Análisis de Patrones Temporales
print("⏰ === ANÁLISIS DE PATRONES TEMPORALES ===\n")

# Patrón por hora del día
print("📊 Generación Promedio por Hora del Día:")
hourly_pattern = pd.DataFrame({
    'Solar': df.groupby('Hora')['Solar_Gen'].mean(),
    'Eolica': df.groupby('Hora')['Eolica_Gen'].mean()
})
print(hourly_pattern.round(2))
print("\n")

# Patrón por mes
print("📅 Generación Promedio por Mes:")
monthly_pattern = pd.DataFrame({
    'Solar': df.groupby('Mes')['Solar_Gen'].mean(),
    'Eolica': df.groupby('Mes')['Eolica_Gen'].mean()
})
print(monthly_pattern.round(2))
print("\n")

# 3. Correlaciones
print("🔗 === ANÁLISIS DE CORRELACIONES ===\n")
correlation_cols = ['Solar_Gen', 'Eolica_Gen', 'Radiacion_Solar', 'Velocidad_Viento', 
                    'Temperatura', 'Hora', 'Demanda']
corr_matrix = df[correlation_cols].corr()
print("Correlación entre variables:")
print(corr_matrix[['Solar_Gen', 'Eolica_Gen']].round(3))
print("\n")

# 4. Análisis de Variabilidad
print("📉 === ANÁLISIS DE VARIABILIDAD ===\n")
print("Coeficiente de Variación (CV = std/mean):")
cv_solar = (df['Solar_Gen'].std() / df['Solar_Gen'].mean() * 100) if df['Solar_Gen'].mean() > 0 else 0
cv_eolica = (df['Eolica_Gen'].std() / df['Eolica_Gen'].mean() * 100) if df['Eolica_Gen'].mean() > 0 else 0
print(f"  Solar: {cv_solar:.2f}%")
print(f"  Eólica: {cv_eolica:.2f}%")
print("\n")

# 5. Visualización del Análisis
print("📊 Generando visualizaciones del análisis...")

fig, axes = plt.subplots(3, 2, figsize=(16, 12))

# Subplot 1: Patrón horario de generación
axes[0, 0].plot(hourly_pattern.index, hourly_pattern['Solar'], 'o-', label='Solar', color='orange', linewidth=2)
axes[0, 0].plot(hourly_pattern.index, hourly_pattern['Eolica'], 's-', label='Eólica', color='purple', linewidth=2)
axes[0, 0].set_title('Patrón Horario de Generación', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Hora del Día')
axes[0, 0].set_ylabel('Generación Promedio (kW)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Subplot 2: Patrón mensual
axes[0, 1].plot(monthly_pattern.index, monthly_pattern['Solar'], 'o-', label='Solar', color='orange', linewidth=2)
axes[0, 1].plot(monthly_pattern.index, monthly_pattern['Eolica'], 's-', label='Eólica', color='purple', linewidth=2)
axes[0, 1].set_title('Patrón Mensual de Generación', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Mes')
axes[0, 1].set_ylabel('Generación Promedio (kW)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Subplot 3: Distribución de generación solar
axes[1, 0].hist(df['Solar_Gen'], bins=50, color='orange', alpha=0.7, edgecolor='black')
axes[1, 0].axvline(df['Solar_Gen'].mean(), color='red', linestyle='--', linewidth=2, label=f'Media: {df["Solar_Gen"].mean():.1f} kW')
axes[1, 0].set_title('Distribución de Generación Solar', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Generación (kW)')
axes[1, 0].set_ylabel('Frecuencia')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Subplot 4: Distribución de generación eólica
axes[1, 1].hist(df['Eolica_Gen'], bins=50, color='purple', alpha=0.7, edgecolor='black')
axes[1, 1].axvline(df['Eolica_Gen'].mean(), color='red', linestyle='--', linewidth=2, label=f'Media: {df["Eolica_Gen"].mean():.1f} kW')
axes[1, 1].set_title('Distribución de Generación Eólica', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Generación (kW)')
axes[1, 1].set_ylabel('Frecuencia')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# Subplot 5: Generación vs. Radiación Solar
axes[2, 0].scatter(df['Radiacion_Solar'], df['Solar_Gen'], alpha=0.3, s=1, color='orange')
axes[2, 0].set_title('Relación: Radiación Solar vs. Generación Solar', fontsize=12, fontweight='bold')
axes[2, 0].set_xlabel('Radiación Solar (W/m²)')
axes[2, 0].set_ylabel('Generación Solar (kW)')
axes[2, 0].grid(True, alpha=0.3)

# Subplot 6: Generación vs. Velocidad del Viento
axes[2, 1].scatter(df['Velocidad_Viento'], df['Eolica_Gen'], alpha=0.3, s=1, color='purple')
axes[2, 1].set_title('Relación: Velocidad del Viento vs. Generación Eólica', fontsize=12, fontweight='bold')
axes[2, 1].set_xlabel('Velocidad del Viento (m/s)')
axes[2, 1].set_ylabel('Generación Eólica (kW)')
axes[2, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Análisis exploratorio completado\n")

## 5. Entrenamiento del Random Forest

In [ ]:
# CELDA 5: Entrenamiento del Random Forest con Features Específicas

print("🌲 Entrenando modelos Random Forest mejorados con features avanzadas...\n")

# Crear features adicionales para mejorar las predicciones
df['Hora_sin'] = np.sin(2 * np.pi * df['Hora'] / 24)  # Ciclo diario
df['Hora_cos'] = np.cos(2 * np.pi * df['Hora'] / 24)
df['Dia_Semana_sin'] = np.sin(2 * np.pi * df['Dia_Semana'] / 7)  # Ciclo semanal
df['Dia_Semana_cos'] = np.cos(2 * np.pi * df['Dia_Semana'] / 7)
df['Mes_sin'] = np.sin(2 * np.pi * df['Mes'] / 12)  # Ciclo anual
df['Mes_cos'] = np.cos(2 * np.pi * df['Mes'] / 12)

# Crear features de lag (valores pasados) - muy importantes para series temporales
print("📊 Creando features de lag y rolling windows...")
for lag in [1, 2, 3, 6, 12, 24]:  # Lags de 1h, 2h, 3h, 12h, 24h
    df[f'Solar_Gen_lag{lag}'] = df['Solar_Gen'].shift(lag).bfill().fillna(0)
    df[f'Eolica_Gen_lag{lag}'] = df['Eolica_Gen'].shift(lag).bfill().fillna(0)
    df[f'Demanda_lag{lag}'] = df['Demanda'].shift(lag).bfill().fillna(0)
    df[f'Radiacion_Solar_lag{lag}'] = df['Radiacion_Solar'].shift(lag).bfill().fillna(0)
    df[f'Velocidad_Viento_lag{lag}'] = df['Velocidad_Viento'].shift(lag).bfill().fillna(0)

# Crear rolling windows (promedios móviles)
for window in [3, 6, 12, 24]:
    df[f'Solar_Gen_rolling{window}'] = df['Solar_Gen'].rolling(window=window, min_periods=1).mean()
    df[f'Eolica_Gen_rolling{window}'] = df['Eolica_Gen'].rolling(window=window, min_periods=1).mean()
    df[f'Demanda_rolling{window}'] = df['Demanda'].rolling(window=window, min_periods=1).mean()

# Features de diferencia (cambio respecto al valor anterior)
df['Solar_Gen_diff'] = df['Solar_Gen'].diff().fillna(0)
df['Eolica_Gen_diff'] = df['Eolica_Gen'].diff().fillna(0)
df['Demanda_diff'] = df['Demanda'].diff().fillna(0)

# Features de ratio (proporciones)
df['Renewable_Ratio'] = (df['Solar_Gen'] + df['Eolica_Gen']) / (df['Demanda'] + 1e-6)
df['Solar_Ratio'] = df['Solar_Gen'] / (df['Demanda'] + 1e-6)
df['Eolica_Ratio'] = df['Eolica_Gen'] / (df['Demanda'] + 1e-6)

# Features específicas para Solar (mejoradas)
features_solar = [
    'Hora', 'Hora_sin', 'Hora_cos',  # Ciclo diario
    'Dia_Semana', 'Dia_Semana_sin', 'Dia_Semana_cos',  # Ciclo semanal
    'Mes', 'Mes_sin', 'Mes_cos',  # Estacionalidad
    'Radiacion_Solar', 'Radiacion_Solar_lag1', 'Radiacion_Solar_lag3', 'Radiacion_Solar_lag6',  # Radiación actual y pasada
    'Temperatura',  # Correlación con radiación
    'Solar_Gen', 'Solar_Gen_lag1', 'Solar_Gen_lag3', 'Solar_Gen_lag6', 'Solar_Gen_lag24',  # Valores pasados
    'Solar_Gen_rolling3', 'Solar_Gen_rolling12', 'Solar_Gen_rolling24',  # Promedios móviles
    'Solar_Gen_diff',  # Cambio respecto al anterior
    'Velocidad_Viento'  # Puede afectar nubosidad
]

# Features específicas para Eólica (mejoradas)
features_eolica = [
    'Hora', 'Hora_sin', 'Hora_cos',  # Ciclo diario
    'Dia_Semana', 'Dia_Semana_sin', 'Dia_Semana_cos',  # Ciclo semanal
    'Mes', 'Mes_sin', 'Mes_cos',  # Estacionalidad
    'Velocidad_Viento', 'Velocidad_Viento_lag1', 'Velocidad_Viento_lag3', 'Velocidad_Viento_lag6',  # Viento actual y pasado
    'Temperatura',  # Puede correlacionarse con viento
    'Eolica_Gen', 'Eolica_Gen_lag1', 'Eolica_Gen_lag3', 'Eolica_Gen_lag6', 'Eolica_Gen_lag24',  # Valores pasados
    'Eolica_Gen_rolling3', 'Eolica_Gen_rolling12', 'Eolica_Gen_rolling24',  # Promedios móviles
    'Eolica_Gen_diff'  # Cambio respecto al anterior
]

# Features para Demanda (mejoradas)
features_demanda = [
    'Hora', 'Hora_sin', 'Hora_cos',
    'Dia_Semana', 'Dia_Semana_sin', 'Dia_Semana_cos',
    'Mes', 'Mes_sin', 'Mes_cos',
    'Temperatura',  # Clima afecta demanda
    'Demanda', 'Demanda_lag1', 'Demanda_lag3', 'Demanda_lag6', 'Demanda_lag24',  # Valores pasados
    'Demanda_rolling3', 'Demanda_rolling12', 'Demanda_rolling24',  # Promedios móviles
    'Demanda_diff',  # Cambio respecto al anterior
    'Solar_Gen', 'Eolica_Gen'  # Generación renovable puede afectar demanda neta
]

# Preparar datos de entrenamiento y validación (80/20 split)
train_size = int(len(df) * 0.8)
train_df = df.iloc[:train_size]
test_df = df.iloc[train_size:]

print(f"📊 División de datos:")
print(f"  Entrenamiento: {len(train_df)} muestras ({len(train_df)/len(df)*100:.1f}%)")
print(f"  Validación: {len(test_df)} muestras ({len(test_df)/len(df)*100:.1f}%)\n")

# Preparar datos para cada modelo
X_solar_train = train_df[features_solar].iloc[:-1]
X_solar_test = test_df[features_solar].iloc[:-1]
y_solar_train = train_df['Solar_Gen'].shift(-1).iloc[:-1]
y_solar_test = test_df['Solar_Gen'].shift(-1).iloc[:-1]

X_eolica_train = train_df[features_eolica].iloc[:-1]
X_eolica_test = test_df[features_eolica].iloc[:-1]
y_eolica_train = train_df['Eolica_Gen'].shift(-1).iloc[:-1]
y_eolica_test = test_df['Eolica_Gen'].shift(-1).iloc[:-1]

X_demanda_train = train_df[features_demanda].iloc[:-1]
X_demanda_test = test_df[features_demanda].iloc[:-1]
y_demanda_train = train_df['Demanda'].shift(-1).iloc[:-1]
y_demanda_test = test_df['Demanda'].shift(-1).iloc[:-1]

# Llenar NaN con medias
y_solar_train = y_solar_train.fillna(y_solar_train.mean())
y_solar_test = y_solar_test.fillna(y_solar_train.mean())
y_eolica_train = y_eolica_train.fillna(y_eolica_train.mean())
y_eolica_test = y_eolica_test.fillna(y_eolica_train.mean())
y_demanda_train = y_demanda_train.fillna(y_demanda_train.mean())
y_demanda_test = y_demanda_test.fillna(y_demanda_train.mean())

# Entrenar modelos mejorados con más árboles y mejores hiperparámetros
print("🌳 Entrenando modelo de DEMANDA (mejorado)...")
rf_demanda = RandomForestRegressor(
    n_estimators=200,  # Más árboles para mejor precisión
    max_depth=20,  # Mayor profundidad
    min_samples_split=3,  # Menos muestras para split
    min_samples_leaf=2,  # Menos muestras por hoja
    max_features='sqrt',  # Features por split
    random_state=42, 
    n_jobs=-1,
    verbose=0
)
rf_demanda.fit(X_demanda_train, y_demanda_train)

print("🌳 Entrenando modelo de GENERACIÓN SOLAR (mejorado)...")
rf_solar = RandomForestRegressor(
    n_estimators=200,
    max_depth=20,
    min_samples_split=3,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=42, 
    n_jobs=-1,
    verbose=0
)
rf_solar.fit(X_solar_train, y_solar_train)

print("🌳 Entrenando modelo de GENERACIÓN EÓLICA (mejorado)...")
rf_eolica = RandomForestRegressor(
    n_estimators=200,
    max_depth=20,
    min_samples_split=3,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=42, 
    n_jobs=-1,
    verbose=0
)
rf_eolica.fit(X_eolica_train, y_eolica_train)

# Evaluar modelos
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_model(model, X_test, y_test, name):
    """Evalúa un modelo y retorna métricas"""
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-6))) * 100  # +1e-6 para evitar div por 0
    
    return {
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2,
        'MAPE': mape
    }

print("\n📊 === EVALUACIÓN DE MODELOS ===\n")

# Evaluar cada modelo
eval_demanda = evaluate_model(rf_demanda, X_demanda_test, y_demanda_test, "Demanda")
eval_solar = evaluate_model(rf_solar, X_solar_test, y_solar_test, "Solar")
eval_eolica = evaluate_model(rf_eolica, X_eolica_test, y_eolica_test, "Eólica")

# Crear tabla de resultados
results_df = pd.DataFrame({
    'Demanda': eval_demanda,
    'Generación Solar': eval_solar,
    'Generación Eólica': eval_eolica
}).T

print("Métricas de Evaluación en Conjunto de Validación:")
print(results_df.round(3))
print("\n")

# Mostrar importancia de features
print("📈 === IMPORTANCIA DE FEATURES ===\n")

print("Top 5 Features para Predicción Solar:")
solar_importance = pd.Series(rf_solar.feature_importances_, index=features_solar).sort_values(ascending=False)
print(solar_importance.head(5).round(4))
print("\n")

print("Top 5 Features para Predicción Eólica:")
eolica_importance = pd.Series(rf_eolica.feature_importances_, index=features_eolica).sort_values(ascending=False)
print(eolica_importance.head(5).round(4))
print("\n")

# Visualizar predicciones vs reales (primeras 168 horas = 1 semana)
print("📊 Generando visualización de predicciones...\n")

fig, axes = plt.subplots(3, 1, figsize=(15, 10))

# Predicción Solar
y_solar_pred = rf_solar.predict(X_solar_test.iloc[:168])
axes[0].plot(range(168), y_solar_test.iloc[:168].values, label='Real', color='blue', linewidth=2)
axes[0].plot(range(168), y_solar_pred, label='Predicción', color='orange', linestyle='--', linewidth=2)
axes[0].set_title('Predicción de Generación Solar (Primera Semana)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Hora')
axes[0].set_ylabel('Generación (kW)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Predicción Eólica
y_eolica_pred = rf_eolica.predict(X_eolica_test.iloc[:168])
axes[1].plot(range(168), y_eolica_test.iloc[:168].values, label='Real', color='blue', linewidth=2)
axes[1].plot(range(168), y_eolica_pred, label='Predicción', color='purple', linestyle='--', linewidth=2)
axes[1].set_title('Predicción de Generación Eólica (Primera Semana)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Hora')
axes[1].set_ylabel('Generación (kW)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Predicción Demanda
y_demanda_pred = rf_demanda.predict(X_demanda_test.iloc[:168])
axes[2].plot(range(168), y_demanda_test.iloc[:168].values, label='Real', color='blue', linewidth=2)
axes[2].plot(range(168), y_demanda_pred, label='Predicción', color='red', linestyle='--', linewidth=2)
axes[2].set_title('Predicción de Demanda (Primera Semana)', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Hora')
axes[2].set_ylabel('Demanda (kW)')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Modelos Random Forest entrenados y evaluados exitosamente")
print(f"📊 Datos de entrenamiento: {len(train_df)-1} muestras")
print(f"🎯 Modelos optimizados con features específicas por tipo de generación")

## 6. Implementación del Entorno de Microrred

In [ ]:
# CELDA 6: Implementación del Entorno de Microrred

class MicrogridEnv(gym.Env):
    """Entorno de microrred híbrida para optimización con RL"""
    
    def __init__(self, df, rf_models, diesel_max=3500.0, batt_cap=1500.0, 
                 diesel_cost=0.55, discount_rate=0.95):
        super(MicrogridEnv, self).__init__()
        
        self.df = df
        self.rf_models = rf_models
        
        # Parámetros del sistema
        self.DIESEL_P_MAX = diesel_max
        self.BATT_CAP_MAX = batt_cap
        self.DIESEL_COST_KWH = diesel_cost
        self.DISCOUNT_RATE = discount_rate
        
        # Estado inicial
        self.current_step = 0
        self.soc = batt_cap / 2.0  # SOC inicial al 50%
        
        # Espacios de acción y observación
        # Acción: [P_diesel, P_batt_charge, P_batt_discharge] (normalizado 0-1)
        self.action_space = spaces.Box(
            low=np.array([0.0, 0.0, 0.0]),
            high=np.array([1.0, 1.0, 1.0]),
            dtype=np.float32
        )
        
        # Observación: [demanda, solar, eolica, soc, hora, dia_semana, temp, rad, viento]
        self.observation_space = spaces.Box(
            low=np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
            high=np.array([3000.0, 1000.0, 500.0, self.BATT_CAP_MAX, 23.0, 6.0, 35.0, 1000.0, 20.0]),
            dtype=np.float32
        )
    
    def reset(self, seed=None, options=None):
        """Reinicia el entorno"""
        super().reset(seed=seed)
        
        self.current_step = 0
        self.soc = self.BATT_CAP_MAX / 2.0
        
        observation = self._get_obs()
        info = {}
        
        return observation, info
    
    def step(self, action):
        """Ejecuta un paso de la simulación"""
        if self.current_step >= len(self.df) - 1:
            return self._get_obs(), 0.0, True, False, {}
        
        # Obtener datos actuales
        row = self.df.iloc[self.current_step]
        P_load = row['Demanda']
        P_solar = row['Solar_Gen']
        P_eolica = row['Eolica_Gen']
        
        # Desnormalizar acciones
        P_diesel = action[0] * self.DIESEL_P_MAX
        P_batt_charge = action[1] * self.BATT_CAP_MAX * 0.5  # Máximo 50% de capacidad por hora
        P_batt_discharge = action[2] * self.BATT_CAP_MAX * 0.5
        
        # Física de la microrred
        P_renewable = P_solar + P_eolica
        P_available = P_diesel + P_renewable
        
        # Gestión de la batería
        if P_batt_charge > 0 and self.soc < self.BATT_CAP_MAX:
            # Cargar batería
            charge_actual = min(P_batt_charge, self.BATT_CAP_MAX - self.soc)
            self.soc += charge_actual
            P_available -= charge_actual
        
        if P_batt_discharge > 0 and self.soc > 0:
            # Descargar batería
            discharge_actual = min(P_batt_discharge, self.soc)
            self.soc -= discharge_actual
            P_available += discharge_actual
        
        # Verificar balance de energía
        if P_available < P_load:
            # Déficit de energía - penalización
            deficit = P_load - P_available
            reward = -deficit * 10.0  # Penalización alta por déficit
        else:
            # Exceso de energía - costo operativo
            excess = P_available - P_load
            cost = P_diesel * self.DIESEL_COST_KWH
            reward = -cost - excess * 0.1  # Pequeña penalización por exceso
        
        # Actualizar paso
        self.current_step += 1
        
        # Verificar si terminó
        done = self.current_step >= len(self.df) - 1
        
        observation = self._get_obs()
        info = {
            'P_load': P_load,
            'P_solar': P_solar,
            'P_eolica': P_eolica,
            'P_diesel': P_diesel,
            'P_available': P_available,
            'SOC': self.soc
        }
        
        return observation, reward, done, False, info
    
    def _get_obs(self):
        """Obtiene la observación actual"""
        if self.current_step >= len(self.df):
            return np.zeros(9, dtype=np.float32)
        
        row = self.df.iloc[self.current_step]
        
        obs = np.array([
            row['Demanda'],
            row['Solar_Gen'],
            row['Eolica_Gen'],
            self.soc,
            row['Hora'],
            row['Dia_Semana'],
            row['Temperatura'],
            row['Radiacion_Solar'],
            row['Velocidad_Viento']
        ], dtype=np.float32)
        
        return obs
    
    def render(self, mode='human'):
        """Renderiza el estado actual"""
        if self.current_step < len(self.df):
            row = self.df.iloc[self.current_step]
            print(f"Paso {self.current_step}: Demanda={row['Demanda']:.1f}kW, "
                  f"Solar={row['Solar_Gen']:.1f}kW, Eólica={row['Eolica_Gen']:.1f}kW, "
                  f"SOC={self.soc:.1f}kWh")

# Crear el entorno
rf_models = {
    'demanda': rf_demanda,
    'solar': rf_solar,
    'eolica': rf_eolica
}

env = MicrogridEnv(df, rf_models, 
                   diesel_max=3500.0, 
                   batt_cap=1500.0, 
                   diesel_cost=0.55)

print("✅ Entorno de microrred creado exitosamente")
print(f"🎯 Espacio de acción: {env.action_space}")
print(f"👁️ Espacio de observación: {env.observation_space}")
print(f"📊 Datos disponibles: {len(df)} horas")

## 7. Entrenamiento del Agente RL (PPO)

In [ ]:
# CELDA 7: Entrenamiento del Agente RL (PPO y DQN)

print("🤖 Iniciando entrenamiento de Agentes RL...")
print("⏱️ Esto puede tomar varios minutos en Google Colab...\n")

# ===== ENTRENAMIENTO PPO =====
print("=" * 60)
print("🚀 ENTRENANDO AGENTE PPO (Proximal Policy Optimization)")
print("=" * 60)

model_ppo = PPO("MlpPolicy", env, verbose=1, 
                gamma=env.DISCOUNT_RATE, 
                learning_rate=3e-4,
                n_steps=1024,  # Reducido para Colab
                batch_size=64,  # Aumentado para mejor estabilidad
                n_epochs=10,
                ent_coef=0.01,  # Entropy coefficient para exploración
                vf_coef=0.5,  # Value function coefficient
                clip_range=0.2)

# Entrenar con timesteps reducidos para Colab
print("\n📚 Entrenando PPO...")
model_ppo.learn(total_timesteps=50000)  # Reducido para Colab

print("✅ Entrenamiento PPO completado")
print("💾 Guardando modelo PPO...")
model_ppo.save("microgrid_ppo_model")
print("✅ Modelo PPO guardado como 'microgrid_ppo_model'\n")

# ===== ENTRENAMIENTO DQN (Q-Learning) =====
print("=" * 60)
print("🎯 ENTRENANDO AGENTE DQN (Deep Q-Network - Q-Learning)")
print("=" * 60)

# DQN requiere un entorno con espacio de acción discreto o Box con discretización
# Vamos a crear una versión discretizada del entorno para DQN
print("\n🔄 Creando entorno discretizado para DQN...")

# Para DQN, discretizamos las acciones en niveles
# Acción: [P_diesel_level, P_batt_charge_level, P_batt_discharge_level]
# Cada nivel va de 0 a 10 (11 niveles por acción)
# Esto da 11^3 = 1331 acciones posibles

class DiscretizedMicrogridEnv(gym.Env):
    """Versión discretizada del entorno para DQN"""
    
    def __init__(self, original_env):
        super(DiscretizedMicrogridEnv, self).__init__()
        self.original_env = original_env
        self.n_levels = 11  # 0-10 niveles por acción
        
        # Espacio de acción discreto: 11^3 = 1331 acciones
        self.action_space = spaces.Discrete(self.n_levels ** 3)
        self.observation_space = original_env.observation_space
        
    def _discretize_action(self, action_idx):
        """Convierte índice de acción discreta a acción continua"""
        # Convertir índice a tres niveles
        level_diesel = action_idx // (self.n_levels ** 2)
        remainder = action_idx % (self.n_levels ** 2)
        level_charge = remainder // self.n_levels
        level_discharge = remainder % self.n_levels
        
        # Normalizar a [0, 1]
        action = np.array([
            level_diesel / (self.n_levels - 1),
            level_charge / (self.n_levels - 1),
            level_discharge / (self.n_levels - 1)
        ], dtype=np.float32)
        
        return action
    
    def reset(self, seed=None, options=None):
        return self.original_env.reset(seed=seed, options=options)
    
    def step(self, action_idx):
        action = self._discretize_action(action_idx)
        return self.original_env.step(action)
    
    def render(self, mode='human'):
        return self.original_env.render(mode)

# Crear entorno discretizado
env_dqn = DiscretizedMicrogridEnv(env)

print(f"✅ Entorno discretizado creado: {env_dqn.action_space.n} acciones posibles\n")

# Entrenar DQN
model_dqn = DQN("MlpPolicy", env_dqn, verbose=1,
                gamma=env.DISCOUNT_RATE,
                learning_rate=1e-4,
                buffer_size=100000,  # Tamaño del replay buffer
                learning_starts=1000,  # Pasos antes de empezar a aprender
                batch_size=32,
                tau=1.0,  # Soft update coefficient
                train_freq=4,  # Frecuencia de entrenamiento
                target_update_interval=1000,  # Actualizar target network cada N pasos
                exploration_fraction=0.1,  # Fracción de exploración
                exploration_initial_eps=1.0,  # Epsilon inicial
                exploration_final_eps=0.05,  # Epsilon final
                max_grad_norm=10)  # Gradient clipping

print("\n📚 Entrenando DQN...")
model_dqn.learn(total_timesteps=50000)  # Mismo número de timesteps que PPO

print("✅ Entrenamiento DQN completado")
print("💾 Guardando modelo DQN...")
model_dqn.save("microgrid_dqn_model")
print("✅ Modelo DQN guardado como 'microgrid_dqn_model'\n")

print("=" * 60)
print("✅ Ambos agentes entrenados exitosamente!")
print("=" * 60)

## 8. Simulación y Comparación de Escenarios

In [ ]:
# CELDA 8: Simulación y Evaluación Comparativa

def run_simulation(env, model=None, is_rl=True, is_dqn=False):
    """Ejecuta una simulación completa"""
    # Obtener el entorno original si es discretizado
    actual_env = env.original_env if hasattr(env, 'original_env') else env
    
    obs, info = actual_env.reset()
    done = False
    total_cost = 0
    total_deficit = 0
    log = []

    while not done:
        idx = actual_env.current_step
        P_load = actual_env.df.iloc[idx]['Demanda']
        P_solar = actual_env.df.iloc[idx]['Solar_Gen']
        P_eolica = actual_env.df.iloc[idx]['Eolica_Gen']

        if is_rl and model is not None:
            if is_dqn:
                # DQN usa acción discreta directamente
                action_idx, _ = model.predict(obs, deterministic=True)
                # Convertir acción discreta a continua
                if hasattr(env, '_discretize_action'):
                    action = env._discretize_action(action_idx)
                else:
                    # Si es el entorno original, necesitamos el wrapper
                    level_diesel = action_idx // 121
                    remainder = action_idx % 121
                    level_charge = remainder // 11
                    level_discharge = remainder % 11
                    action = np.array([
                        level_diesel / 10.0,
                        level_charge / 10.0,
                        level_discharge / 10.0
                    ], dtype=np.float32)
                obs, reward, done, _, info = actual_env.step(action)
            else:
                # PPO usa acción continua directamente
                action, _ = model.predict(obs, deterministic=True)
                obs, reward, done, _, info = actual_env.step(action)
            
            costo_paso = -reward  # El reward es negativo del costo
            P_diesel_usado = info.get('P_diesel', 0)
            P_available = info.get('P_available', 0)
            deficit = max(0, P_load - P_available)
            total_deficit += deficit
        else:
            # Escenario Línea Base (Solo Diésel)
            P_req = P_load
            P_diesel_usado = P_req
            costo_paso = P_diesel_usado * actual_env.DIESEL_COST_KWH
            
            # Simulación de paso
            actual_env.current_step += 1
            done = actual_env.current_step >= len(actual_env.df) - 1
            obs = actual_env._get_obs()
            deficit = 0

        total_cost += costo_paso
        log.append({
            'Tiempo': actual_env.df.index[idx] if hasattr(actual_env.df, 'index') else idx,
            'Demanda': P_load,
            'Solar_Gen': P_solar,
            'Eolica_Gen': P_eolica,
            'P_Diesel_Usado': P_diesel_usado,
            'Costo': costo_paso,
            'SOC': actual_env.soc if is_rl else (actual_env.BATT_CAP_MAX / 2.0 if hasattr(actual_env, 'BATT_CAP_MAX') else 0),
            'Deficit': deficit
        })

    return pd.DataFrame(log), total_cost, total_deficit

# Ejecutar todas las simulaciones
print("=" * 60)
print("🔄 EJECUTANDO SIMULACIONES")
print("=" * 60)

print("\n1️⃣ Ejecutando simulación Base (Solo Diésel)...")
log_base, cost_base, deficit_base = run_simulation(env, model=None, is_rl=False)

print("2️⃣ Ejecutando simulación PPO-Híbrida...")
log_ppo, cost_ppo, deficit_ppo = run_simulation(env, model_ppo, is_rl=True, is_dqn=False)

print("3️⃣ Ejecutando simulación DQN-Híbrida (Q-Learning)...")
log_dqn, cost_dqn, deficit_dqn = run_simulation(env_dqn, model_dqn, is_rl=True, is_dqn=True)

print("\n" + "=" * 60)
print("📊 === RESULTADOS COMPARATIVOS DE LA SIMULACIÓN ===")
print("=" * 60)
print(f"\n💰 Costo Total Base (Solo Diésel):     ${cost_base:,.2f}")
print(f"💰 Costo Total PPO (RL-Híbrido):        ${cost_ppo:,.2f}")
print(f"💰 Costo Total DQN (Q-Learning):       ${cost_dqn:,.2f}")

print(f"\n💵 Ahorro PPO vs Base:                 ${cost_base - cost_ppo:,.2f} ({(cost_base - cost_ppo) / cost_base * 100:.2f}%)")
print(f"💵 Ahorro DQN vs Base:                 ${cost_base - cost_dqn:,.2f} ({(cost_base - cost_dqn) / cost_base * 100:.2f}%)")
print(f"💵 Diferencia PPO vs DQN:              ${abs(cost_ppo - cost_dqn):,.2f}")

print(f"\n⚠️  Déficit Energético Base:           {deficit_base:.2f} kWh")
print(f"⚠️  Déficit Energético PPO:            {deficit_ppo:.2f} kWh")
print(f"⚠️  Déficit Energético DQN:            {deficit_dqn:.2f} kWh")

# Determinar mejor modelo
best_model = "PPO" if cost_ppo < cost_dqn else "DQN"
best_cost = min(cost_ppo, cost_dqn)
print(f"\n🏆 Mejor Modelo: {best_model} con costo de ${best_cost:,.2f}")
print("=" * 60)

## 9. Análisis de Resultados y Visualización

In [ ]:
# CELDA 9: Visualización Comparativa Mejorada

print("📊 Generando visualizaciones comparativas...")

# Configurar matplotlib para Colab
plt.style.use('default')
plt.rcParams['figure.figsize'] = (18, 14)

# Crear figura con más subplots para comparación completa
fig = plt.figure(figsize=(18, 14))

# Subplot 1: Comparación de Uso de Diésel (3 modelos)
plt.subplot(3, 3, 1)
# Mostrar solo una muestra para mejor visualización
sample_size = min(500, len(log_base))
indices = np.linspace(0, len(log_base)-1, sample_size, dtype=int)
plt.plot(log_base.iloc[indices]['Tiempo'], log_base.iloc[indices]['P_Diesel_Usado'], 
         label='Base (Solo Diésel)', color='red', linestyle='--', alpha=0.7, linewidth=1.5)
plt.plot(log_ppo.iloc[indices]['Tiempo'], log_ppo.iloc[indices]['P_Diesel_Usado'], 
         label='PPO-Híbrido', color='blue', alpha=0.7, linewidth=1.5)
plt.plot(log_dqn.iloc[indices]['Tiempo'], log_dqn.iloc[indices]['P_Diesel_Usado'], 
         label='DQN-Híbrido (Q-Learning)', color='green', alpha=0.7, linewidth=1.5)
plt.title('Comparación de Uso de Generación Diésel', fontweight='bold')
plt.xlabel('Tiempo')
plt.ylabel('Potencia Diésel (kW)')
plt.legend(fontsize=8)
plt.grid(True, alpha=0.3)

# Subplot 2: Estado de Carga (SOC) - PPO
plt.subplot(3, 3, 2)
plt.plot(log_ppo.iloc[indices]['Tiempo'], log_ppo.iloc[indices]['SOC'], 
         label='SOC (PPO)', color='blue', linewidth=1.5)
plt.title('Estado de Carga (SOC) - PPO', fontweight='bold')
plt.xlabel('Tiempo')
plt.ylabel('SOC (kWh)')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 3: Estado de Carga (SOC) - DQN
plt.subplot(3, 3, 3)
plt.plot(log_dqn.iloc[indices]['Tiempo'], log_dqn.iloc[indices]['SOC'], 
         label='SOC (DQN)', color='green', linewidth=1.5)
plt.title('Estado de Carga (SOC) - DQN', fontweight='bold')
plt.xlabel('Tiempo')
plt.ylabel('SOC (kWh)')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 4: Generación Renovable
plt.subplot(3, 3, 4)
plt.plot(log_ppo.iloc[indices]['Tiempo'], log_ppo.iloc[indices]['Solar_Gen'], 
         label='Solar', color='orange', alpha=0.7, linewidth=1.5)
plt.plot(log_ppo.iloc[indices]['Tiempo'], log_ppo.iloc[indices]['Eolica_Gen'], 
         label='Eólica', color='purple', alpha=0.7, linewidth=1.5)
plt.title('Generación Renovable', fontweight='bold')
plt.xlabel('Tiempo')
plt.ylabel('Potencia (kW)')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 5: Comparación de Costos (Barras)
plt.subplot(3, 3, 5)
costos = [cost_base, cost_ppo, cost_dqn]
labels = ['Solo Diésel', 'PPO', 'DQN']
colors = ['red', 'blue', 'green']
bars = plt.bar(labels, costos, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
plt.title('Comparación de Costos Totales', fontweight='bold')
plt.ylabel('Costo Total (USD)')
plt.grid(True, alpha=0.3, axis='y')

# Agregar valores en las barras
for i, (bar, v) in enumerate(zip(bars, costos)):
    plt.text(bar.get_x() + bar.get_width()/2, v + max(costos) * 0.01, 
             f'${v:,.0f}', ha='center', va='bottom', fontweight='bold')

# Subplot 6: Ahorro Comparativo
plt.subplot(3, 3, 6)
ahorros = [
    (cost_base - cost_ppo) / cost_base * 100,
    (cost_base - cost_dqn) / cost_base * 100
]
labels_ahorro = ['PPO', 'DQN']
colors_ahorro = ['blue', 'green']
bars_ahorro = plt.bar(labels_ahorro, ahorros, color=colors_ahorro, alpha=0.7, 
                      edgecolor='black', linewidth=1.5)
plt.title('Porcentaje de Ahorro vs Base', fontweight='bold')
plt.ylabel('Ahorro (%)')
plt.grid(True, alpha=0.3, axis='y')
for bar, v in zip(bars_ahorro, ahorros):
    plt.text(bar.get_x() + bar.get_width()/2, v + max(ahorros) * 0.01, 
             f'{v:.2f}%', ha='center', va='bottom', fontweight='bold')

# Subplot 7: Distribución de Costos por Hora (Boxplot)
plt.subplot(3, 3, 7)
cost_data = [log_base['Costo'].values, log_ppo['Costo'].values, log_dqn['Costo'].values]
bp = plt.boxplot(cost_data, labels=['Base', 'PPO', 'DQN'], patch_artist=True)
for patch, color in zip(bp['boxes'], ['red', 'blue', 'green']):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
plt.title('Distribución de Costos por Hora', fontweight='bold')
plt.ylabel('Costo (USD)')
plt.grid(True, alpha=0.3, axis='y')

# Subplot 8: Comparación de Déficit Energético
plt.subplot(3, 3, 8)
deficits = [deficit_base, deficit_ppo, deficit_dqn]
bars_deficit = plt.bar(labels, deficits, color=colors, alpha=0.7, 
                       edgecolor='black', linewidth=1.5)
plt.title('Déficit Energético Total', fontweight='bold')
plt.ylabel('Déficit (kWh)')
plt.grid(True, alpha=0.3, axis='y')
for bar, v in zip(bars_deficit, deficits):
    if v > 0:
        plt.text(bar.get_x() + bar.get_width()/2, v + max(deficits) * 0.01, 
                 f'{v:.1f}', ha='center', va='bottom', fontweight='bold')

# Subplot 9: Comparación de SOC Promedio
plt.subplot(3, 3, 9)
soc_means = [
    log_ppo['SOC'].mean(),
    log_dqn['SOC'].mean()
]
soc_stds = [
    log_ppo['SOC'].std(),
    log_dqn['SOC'].std()
]
bars_soc = plt.bar(['PPO', 'DQN'], soc_means, yerr=soc_stds, 
                   color=['blue', 'green'], alpha=0.7, 
                   edgecolor='black', linewidth=1.5, capsize=5)
plt.title('SOC Promedio ± Desviación', fontweight='bold')
plt.ylabel('SOC (kWh)')
plt.grid(True, alpha=0.3, axis='y')
for bar, v in zip(bars_soc, soc_means):
    plt.text(bar.get_x() + bar.get_width()/2, v + soc_stds[labels_ahorro.index(bar.get_x())] + 10, 
             f'{v:.1f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Estadísticas adicionales
print("\n" + "=" * 60)
print("📈 === RESUMEN DETALLADO DE RESULTADOS ===")
print("=" * 60)

ahorro_ppo = (cost_base - cost_ppo) / cost_base * 100
ahorro_dqn = (cost_base - cost_dqn) / cost_base * 100

print(f"\n🎯 PPO logra una reducción del {ahorro_ppo:.2f}% en costos operativos.")
print(f"💰 Ahorro PPO estimado: ${(cost_base - cost_ppo):,.2f}")

print(f"\n🎯 DQN logra una reducción del {ahorro_dqn:.2f}% en costos operativos.")
print(f"💰 Ahorro DQN estimado: ${(cost_base - cost_dqn):,.2f}")

print(f"\n🔋 Estadísticas del SOC - PPO:")
print(f"📊 SOC promedio: {log_ppo['SOC'].mean():.1f} kWh")
print(f"📉 SOC mínimo: {log_ppo['SOC'].min():.1f} kWh")
print(f"📈 SOC máximo: {log_ppo['SOC'].max():.1f} kWh")
print(f"📊 Desviación estándar: {log_ppo['SOC'].std():.1f} kWh")

print(f"\n🔋 Estadísticas del SOC - DQN:")
print(f"📊 SOC promedio: {log_dqn['SOC'].mean():.1f} kWh")
print(f"📉 SOC mínimo: {log_dqn['SOC'].min():.1f} kWh")
print(f"📈 SOC máximo: {log_dqn['SOC'].max():.1f} kWh")
print(f"📊 Desviación estándar: {log_dqn['SOC'].std():.1f} kWh")

print(f"\n⚡ Eficiencia Energética:")
print(f"📊 Uso promedio diésel Base: {log_base['P_Diesel_Usado'].mean():.1f} kW")
print(f"📊 Uso promedio diésel PPO: {log_ppo['P_Diesel_Usado'].mean():.1f} kW")
print(f"📊 Uso promedio diésel DQN: {log_dqn['P_Diesel_Usado'].mean():.1f} kW")

print(f"\n✅ Simulación comparativa completada exitosamente!")
print("=" * 60)